# Fetch Image FIles

In [8]:
import sqlite3
DB_FILE = '020-sqlite/ogham.db'
conn = sqlite3.connect(DB_FILE)

SQL = f"""
SELECT DISTINCT SITE,STONE,INSCRIPTION
FROM INSCRIP
"""
cursor = conn.cursor()
rows = cursor.execute(SQL).fetchall()


In [20]:
urls = []
PREFIX = "https://www.ucl.ac.uk/archaeology/cisp/database"
for site,stone,inscription in rows:
    urls += [((site,stone,inscription),
             f"{PREFIX}/picpage/{site.lower()}_{stone}_{inscription}.html")]


In [27]:
import requests
import os
HTML_DIR = '030 - image html files'
for (site,stone,inscription),url in urls:
    of_name = f"{HTML_DIR}/{site}_{stone}_{inscription}.html"
    if os.path.exists(of_name):
        continue
    try:
        r = requests.get(url)
    except:
        continue
    of = open(of_name, 'w')
    of.write(r.text)
    of.flush()

In [36]:
import os
image_urls = []
for fn in os.listdir(HTML_DIR):
    full_name = os.path.join(HTML_DIR, fn)
    image_urls += [(fn,l) for l in open(full_name).readlines()
                  if "img src" in l and "images" in l and "close" not in l
                  and "mob-nav" not in l]


In [37]:
inscriptions_images = []
for fn, im in image_urls:
    site,stone,inscription = fn[:-5].split('_')
    img_url = im.split("\"")[1]
    inscriptions_images += [(site,stone,inscription,img_url)]


In [45]:
import shutil
IMAGES_DIR = '040 - images'
for site,stone,inscription,img_url in inscriptions_images:
    r = requests.get(f"{PREFIX}/picpage/{img_url}", stream=True)
    if r.status_code != 200:
        continue
    
    on_prefix = os.path.join(IMAGES_DIR,f"{site}_{stone}_{inscription}")
    i = 0
    on_suffix = f"_{i}.{img_url[-3:]}"
    while os.path.exists(on_prefix+on_suffix):
        i+=1
        on_suffix = f"_{i}.{img_url[-3:]}"
    on = on_prefix+on_suffix
    
    with open(on, 'wb') as f:
        r.raw.decode_content = True
        shutil.copyfileobj(r.raw, f)   